In [1]:
import os
import re
import pandas as pd
pd.set_option('display.max_colwidth', 200)

import tarfile
import scipy.io
from collections import defaultdict

import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
# For tf.dataset
AUTO = tf.data.experimental.AUTOTUNE

In [2]:
train = pd.read_csv('../input/fgvc-aircraft/train.csv')
val = pd.read_csv('../input/fgvc-aircraft/val.csv')
test = pd.read_csv('../input/fgvc-aircraft/test.csv')
train.head()

,filename,Classes,Labels
0,1025794.jpg,707-320,0
1,1340192.jpg,707-320,0
2,0056978.jpg,707-320,0
3,0698580.jpg,707-320,0
4,0450014.jpg,707-320,0


In [3]:
tpath = "../input/fgvc-aircraft/fgvc-aircraft-2013b/fgvc-aircraft-2013b/data/images"
train_paths = train.filename.apply(lambda x: os.path.join(tpath, x))
train_paths.values

array(['../input/fgvc-aircraft/fgvc-aircraft-2013b/fgvc-aircraft-2013b/data/images/1025794.jpg',
       '../input/fgvc-aircraft/fgvc-aircraft-2013b/fgvc-aircraft-2013b/data/images/1340192.jpg',
       '../input/fgvc-aircraft/fgvc-aircraft-2013b/fgvc-aircraft-2013b/data/images/0056978.jpg',
       ...,
       '../input/fgvc-aircraft/fgvc-aircraft-2013b/fgvc-aircraft-2013b/data/images/0472681.jpg',
       '../input/fgvc-aircraft/fgvc-aircraft-2013b/fgvc-aircraft-2013b/data/images/1597829.jpg',
       '../input/fgvc-aircraft/fgvc-aircraft-2013b/fgvc-aircraft-2013b/data/images/0197891.jpg'],
      dtype=object)

In [4]:
from tensorflow.keras.utils import to_categorical

# convert to one-hot-encoding-labels
train_labels = to_categorical(train.Labels)
train_labels.shape

(3334, 100)

In [5]:
tpath = "../input/fgvc-aircraft/fgvc-aircraft-2013b/fgvc-aircraft-2013b/data/images"
val_paths = val.filename.apply(lambda x: os.path.join(tpath, x))
val_paths.values

array(['../input/fgvc-aircraft/fgvc-aircraft-2013b/fgvc-aircraft-2013b/data/images/0481847.jpg',
       '../input/fgvc-aircraft/fgvc-aircraft-2013b/fgvc-aircraft-2013b/data/images/0810608.jpg',
       '../input/fgvc-aircraft/fgvc-aircraft-2013b/fgvc-aircraft-2013b/data/images/1514481.jpg',
       ...,
       '../input/fgvc-aircraft/fgvc-aircraft-2013b/fgvc-aircraft-2013b/data/images/1340345.jpg',
       '../input/fgvc-aircraft/fgvc-aircraft-2013b/fgvc-aircraft-2013b/data/images/0765754.jpg',
       '../input/fgvc-aircraft/fgvc-aircraft-2013b/fgvc-aircraft-2013b/data/images/1594714.jpg'],
      dtype=object)

In [6]:
# convert to one-hot-encoding-labels
val_labels = to_categorical(val.Labels)
val_labels.shape

(3333, 100)

In [7]:
tpath = "../input/fgvc-aircraft/fgvc-aircraft-2013b/fgvc-aircraft-2013b/data/images"
test_paths = test.filename.apply(lambda x: os.path.join(tpath, x))
test_paths.values

array(['../input/fgvc-aircraft/fgvc-aircraft-2013b/fgvc-aircraft-2013b/data/images/1514522.jpg',
       '../input/fgvc-aircraft/fgvc-aircraft-2013b/fgvc-aircraft-2013b/data/images/0747566.jpg',
       '../input/fgvc-aircraft/fgvc-aircraft-2013b/fgvc-aircraft-2013b/data/images/1008575.jpg',
       ...,
       '../input/fgvc-aircraft/fgvc-aircraft-2013b/fgvc-aircraft-2013b/data/images/0329381.jpg',
       '../input/fgvc-aircraft/fgvc-aircraft-2013b/fgvc-aircraft-2013b/data/images/0523192.jpg',
       '../input/fgvc-aircraft/fgvc-aircraft-2013b/fgvc-aircraft-2013b/data/images/0810303.jpg'],
      dtype=object)

In [8]:
def decode_image(filename, label=None, image_size=(299, 299)):
    bits = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(bits, channels=3)
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.image.resize(image, image_size)
    
    if label is None:
        return image
    else:
        return image, label

In [9]:
def augment(image, label=None):
    image = tf.image.random_flip_left_right(image)
    
    if label is None:
        return image
    else:
        return image, label

In [10]:
batch_size = 32

train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((train_paths, train_labels))
    .map(decode_image, num_parallel_calls=AUTO)
    .map(augment, num_parallel_calls=AUTO)
    .repeat()
    .shuffle(2048)
    .batch(batch_size)
    .prefetch(AUTO)
)

val_dataset = (tf.data.Dataset
        .from_tensor_slices((val_paths, val_labels))
        .map(decode_image, num_parallel_calls=AUTO)
        .batch(batch_size)
)

test_dataset = (tf.data.Dataset
        .from_tensor_slices(test_paths)
        .map(decode_image, num_parallel_calls=AUTO)
        .batch(batch_size)
)

In [11]:
from tensorflow.keras.applications import ResNet101
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

# from efficientnet.tfkeras import EfficientNetB2

model = tf.keras.Sequential([
        ResNet101(weights = 'imagenet', 
                       include_top = False,
                       input_shape = (299, 299, 3)),
        GlobalAveragePooling2D(),
        Dense(1024, activation = 'relu'),
        Dropout(0.25),
        Dense(512, activation = 'relu'),
        Dropout(0.25),
        Dense(100, activation = 'softmax')
])
    
model.compile(optimizer = 'sgd',
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])
    
model.summary()

171450368/171446536 [==============================] - 3s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet101 (Model)            (None, 10, 10, 2048)      42658176  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 1024)              2098176   
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_______________________________________________________

In [12]:
def Train_model(model, batch_size, EPOCHS):
    n_steps = train_labels.shape[0] // batch_size
    EPOCHS = EPOCHS
    
    Model = model
    history = Model.fit(train_dataset, 
                    steps_per_epoch = n_steps,
                    epochs = EPOCHS,
                    validation_data = val_dataset,
                    verbose = 1)
    return Model

In [13]:
batch_size = 32
EPOCHS = 50

def check(x, y):
    if x == y:
        return 1
    else:
        return 0
    
print('Training')
model = Train_model(model, batch_size, EPOCHS)
pred = model.predict(test_dataset, verbose=1)
    
test['Prediction'] = pred.argmax(axis=1)
test['Score'] = test.apply(lambda x: check(x['Prediction'], x['Labels']), axis=1)
print('Test accuracy on iterations is ', 100 * test.Score.sum() / test.shape[0])

Training
Epoch 1/50
104/104 [==============================] - 89s 852ms/step - loss: 4.5259 - accuracy: 0.0421 - val_loss: 4.6116 - val_accuracy: 0.0396
Epoch 2/50
104/104 [==============================] - 85s 815ms/step - loss: 4.0449 - accuracy: 0.1106 - val_loss: 4.6696 - val_accuracy: 0.0099
Epoch 3/50
104/104 [==============================] - 85s 817ms/step - loss: 3.3164 - accuracy: 0.2215 - val_loss: 4.7139 - val_accuracy: 0.0120
Epoch 4/50
104/104 [==============================] - 86s 825ms/step - loss: 2.4632 - accuracy: 0.3900 - val_loss: 4.7622 - val_accuracy: 0.0147
Epoch 5/50
104/104 [==============================] - 84s 812ms/step - loss: 1.8267 - accuracy: 0.5108 - val_loss: 4.6597 - val_accuracy: 0.0159
Epoch 6/50
104/104 [==============================] - 84s 808ms/step - loss: 1.3455 - accuracy: 0.6400 - val_loss: 4.2390 - val_accuracy: 0.0477
Epoch 7/50
104/104 [==============================] - 86s 822ms/step - loss: 0.9722 - accuracy: 0.7290 - val_loss: 3.1462

In [14]:
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(test.Labels, test.Prediction))

              precision    recall  f1-score   support

           0       0.59      0.67      0.63        33
           1       0.75      0.73      0.74        33
           2       0.73      0.65      0.69        34
           3       0.37      0.30      0.33        33
           4       0.64      0.42      0.51        33
           5       0.56      0.53      0.55        34
           6       0.66      0.76      0.70        33
           7       0.39      0.33      0.36        33
           8       0.40      0.50      0.44        34
           9       0.56      0.58      0.57        33
          10       0.32      0.39      0.35        33
          11       0.37      0.32      0.34        34
          12       0.41      0.39      0.40        33
          13       0.32      0.30      0.31        33
          14       0.48      0.44      0.46        34
          15       0.69      0.76      0.72        33
          16       0.53      0.55      0.54        33
          17       0.40    